In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
file = "https://raw.githubusercontent.com/Nightshade96/DataSets/main/Store_Id_finder.csv"

In [3]:
store_id_reader = pd.read_csv(file,header=0,keep_default_na=True)

In [4]:
first_column = store_id_reader.iloc[:, 0]

lets try some pre processing for data like
 NNT FAMOUSFOOTWEAR#570903 where number is attached with the words as well


NNT FAMOUS FOOTWEAR572264 has a word which is not seperated by #, so we will explore the optionof regex for such words in order to get them into order.

In [ ]:
work for this file, create new train & validate data

In [6]:
def text_num_split(item):
    for index, letter in enumerate(item, 0):
        if letter.isdigit():
            a = [item[:index],item[index:]]
            return a

In [27]:
first_column_edited_again = []
for i in first_column:
  res = text_num_split(i)
  res = ' '.join(res)
  first_column_edited_again.append(res)

In [29]:
  first_column_edited_again


['DOLRTREE  2257 00022574 ROSWELL',
 'AUTOZONE # 3547',
 'TGI FRIDAYS  1485 0000',
 'BUFFALO WILD WINGS  003',
 'J. CREW # 568 0',
 'KRISPY KREME  0040        GREENVILLE   SC',
 'FIVE GUYS MN  1847 ECOM   612-339-9733 MN',
 'CASEYS GEN STORE  2650',
 'HUDDLE HOUSE  535',
 'JAMBA JUICE # 1305',
 'ANN TAYLOR FACTORY # 2202',
 'Subway  26824',
 'MARSHALLS # 0688',
 'OREILLY AUTO # 4681',
 'TA # 227 BARSTOW REST',
 'SONIC # 3207',
 'HY VEE  1040',
 "MCDONALD'S F 1013",
 'CHEVRON  00207812',
 'EXPRESS# 0920',
 "MCDONALD'S F 16829",
 'CHEVRON  0208998    Q61',
 'STARBUCKS STORE  49134',
 'CHEVRON  0302900',
 'CIRCLE K  01251',
 'NST BEST BUY # 479  002610',
 'HOLIDAY STNSTORE  0354',
 "DOMINO'S  7430",
 'NST ROSS STORE # 483280353',
 "MCDONALD'S F 5579",
 'NST ROSS STORES # 10262660',
 'HOLIDAY STNSTORE  0447',
 'NST BEST BUY # 25   232381',
 'MEIJER #  196',
 'SUBWAY         00516260',
 "DOMINO'S  6870",
 'MARATHON PETRO 045518',
 "MCDONALD'S F 2128",
 'DOLLAR GENERAL # 22357',
 'SUNOCO  07

Explanation:

\d*- matches 0 or more occurrence digits
\s* - matches 0 or more occurrence whitespace after the number
[-\\.]? - matches 0 or 1 occurrence of '-' '.' '\'
\s* - matches 0 or more occurrence whitespace after that
([a-zA-Z\s*]+) -matches any characters after that (items you need to extract)

In [8]:
#file will be stored in /content/training_data.txt   &     /content/validation_data.txt
#download to local as we have to use it for tagging and annontation purposes.
with open('validation_data.txt', 'w') as f:
    for item in first_column_edited_again[100:200]:
        f.write("%s\n" % item)

In [9]:
! pip install -U spacy -q

In [10]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.2.4                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        
Pipelines                                      



In [11]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [12]:
import json
t = open('/content/training_data.json')
v = open('/content/validation_data.json')
TRAIN_DATA = json.load(t)
VALIDATION_DATA = json.load(v)

In [13]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = [] 
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 100/100 [00:00<00:00, 804.12it/s]


In [14]:
for text, annot in tqdm(VALIDATION_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./validation_data.spacy") # save the docbin object

100%|██████████| 146/146 [00:00<00:00, 913.21it/s]


In [15]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [16]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./validation_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-04-09 03:01:17,502] [INFO] Set up nlp object from config
[2022-04-09 03:01:17,514] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-04-09 03:01:17,519] [INFO] Created vocabulary
[2022-04-09 03:01:17,520] [INFO] Finished initializing nlp object
[2022-04-09 03:01:17,747] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     65.67    0.00    0.00    0.00    0.00
 44     200          8.13    740.37   94.14   90.60   97.97    0.94
 95     400          0.00      0.00   93.96   90.26   97.97    0.94
162     600          0.00      0.00   93.73   90

In [17]:
nlp_ner = spacy.load("/content/model-best")

In [22]:
ents = []
#picking up last 100, which are testing data
for i in first_column_edited_again[-100:]:
  doc1 = nlp_ner(i)
  ents.append(doc1.ents)

results =  pd.DataFrame({'transaction_descriptor':first_column[-100:],'store_number':ents})

In [23]:
results = results.reset_index(drop=True)
results.head(10)

,transaction_descriptor,store_number
0,IN-N-OUT BURGER #242,"((242),)"
1,BP#9442088LIBERTYVILLE B,()
2,JCPENNEY 1419,"((1419),)"
3,ROSS STORES #1019,"((1019),)"
4,WM SUPERCENTER #38,"((38),)"
5,TUESDAY MORNING # 0673 06,"((0673), (06))"
6,IHOP 629 WHITE HOUSE TN,"((629),)"
7,LBOUTLETS#4249 1475 N BUR,"((4249), (1475))"
8,"WINN DIXIE #2505 VALRICO, FL (3454)","((2505), (3454))"
9,BURLINGTON STORES 825,"((825),)"


In [20]:
#post processing for 
results['store_number'][0][0]

242

In [26]:
for i in first_column_edited_again[-100:]:
  doc1 = nlp_ner(i)
  spacy.displacy.render(doc1, style="ent", jupyter=True) # display in Jupyter

/usr/local/lib/python3.7/dist-packages/spacy/displacy/__init__.py:200: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
